In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import optim
from torch.optim import lr_scheduler

import transformers
from transformers.optimization import Adafactor, AdafactorSchedule

from sklearn.metrics import fbeta_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import os
import random
import time
from tqdm.notebook import tqdm
import datetime as dt
import copy
import matplotlib.pyplot as plt


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]='0'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

In [3]:
model_name_dict = {
    "PubMedBERT": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
}

class Hparams:
    def __init__(self):
        self.random_seed = 0 # BAD:2021
        self.data_dir = './data'
        self.output_dir = './outputs'
        self.batch_size = 128
        self.token_max_length = 512
        self.model_name = model_name_dict['PubMedBERT']
        self.num_epochs = 5
        self.class_1_weight = 150
        self.initial_lr = 2e-5  # 2e-5
        self.model_type = 'lstm_ex'  # cnn, lstm, lstm_ex
        self.upsample_pos_n = 1
        self.use_col = 'title_abstract'  # title, abstract, title_abstract
        self.train_argument = True
        self.test_size = 0.0  # 0.2
        self.cv_n = 5
    

hps = Hparams()


def seed_torch(seed:int):
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(hps.random_seed)

## DataFrame

In [43]:
orig_df = pd.read_csv(os.path.join(hps.data_dir, 'train.csv'), index_col=0)
submit_df = pd.read_csv(os.path.join(hps.data_dir, 'test.csv'), index_col=0)
sample_submit_df = pd.read_csv(os.path.join(hps.data_dir, 'sample_submit.csv'), index_col=0, header=None, names=['judgement'])

# 修正
orig_df.loc[2488, 'judgement'] = 0
orig_df.loc[7708, 'judgement'] = 0

# 補完
orig_df['abstract'].fillna('', inplace=True)
orig_df['title_abstract'] = orig_df.title + orig_df.abstract

submit_df['abstract'].fillna('', inplace=True)
submit_df['title_abstract'] = submit_df.title + submit_df.abstract
submit_df['judgement'] = -1
submit_df.reset_index(inplace=True, drop=False)

## Cross Validations SetUp

In [6]:
if hps.test_size > 0.0:
    train_df, test_df = train_test_split(orig_df, test_size=hps.test_size, random_state=hps.random_seed, shuffle=True, stratify=orig_df.judgement)
else:
    train_df = orig_df.copy()
    test_df = orig_df.copy()

In [7]:
def get_cv_number(df, cv_n):

    df['cv_id'] = 0

    neg_idx = df.loc[df.judgement==0].index.tolist()
    pos_idx = df.loc[df.judgement==1].index.tolist()

    neg_idx = [list(a) for a in list(np.array_split(random.sample(neg_idx, len(neg_idx)), cv_n))]
    pos_idx = [list(a) for a in list(np.array_split(random.sample(pos_idx, len(pos_idx)), cv_n))]

    for i in range(cv_n):
        n_id = neg_idx[i]
        p_id = pos_idx[i]
        df.loc[n_id, 'cv_id'] = i
        df.loc[p_id, 'cv_id'] = i

    df = df.sort_index()

    for i in range(cv_n):
        tmp_df = df.loc[df.cv_id==i]
        print('cv_id:', i, '->  pos:', len(tmp_df.loc[tmp_df.judgement==1]), ' / neg:', len(tmp_df.loc[tmp_df.judgement==0]), ' / all:', len(tmp_df))
        
    return df


train_df = get_cv_number(train_df, cv_n=hps.cv_n)

cv_id: 0 ->  pos: 126  / neg: 5303  / all: 5429
cv_id: 1 ->  pos: 126  / neg: 5303  / all: 5429
cv_id: 2 ->  pos: 126  / neg: 5303  / all: 5429
cv_id: 3 ->  pos: 126  / neg: 5303  / all: 5429
cv_id: 4 ->  pos: 126  / neg: 5303  / all: 5429


## Hugging Face

In [8]:
base_tokenizer = transformers.AutoTokenizer.from_pretrained(hps.model_name)

bert_config = transformers.AutoConfig.from_pretrained(hps.model_name)
bert_config.output_hidden_states = True

## DataSet / DataLoader

In [9]:
class TextClassificationDataset(Dataset):
    def __init__(self, df, tokenizer, use_col='title_abstract', token_max_length=512, argument=False, upsample_pos_n=1):

        if upsample_pos_n > 1:
            df_pos = df.loc[df.judgement==1]
            df_pos = pd.concat([df_pos for i in range(int(upsample_pos_n))], axis=0).reset_index(drop=True)
            df_neg = df.loc[df.judgement==0]
            self.df = pd.concat([df_pos, df_neg], axis=0).reset_index(drop=True)
        else:
            self.df = df
        
        self.tokenizer = tokenizer
        self.argument = argument
        self.use_col = use_col

    def text_argument(self, text, drop_min_seq=3, seq_sort=True):
        seq_list = text.split('. ')
        seq_len = len(seq_list)
        if seq_len >= drop_min_seq:
            orig_idx_list = list(range(0, seq_len))
            idx_list = random.sample(orig_idx_list, random.randint(round(seq_len * 0.7), seq_len))
            if seq_sort:
                idx_list = sorted(idx_list)
            insert_idx_list = random.sample(orig_idx_list, random.randint(0, seq_len//3))
            for x in insert_idx_list:
                idx = random.randint(0, len(idx_list))
                idx_list.insert(idx, x)
            seq_list = [seq_list[i] for i in idx_list]
        text = '. '.join(seq_list)
        return text

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        text = self.df.loc[idx, self.use_col]

        if self.argument:
            text = self.text_argument(text, drop_min_seq=3, seq_sort=True)

        token = self.tokenizer.encode_plus(
            text,
            padding = 'max_length', max_length = hps.token_max_length, truncation = True,
            return_attention_mask=True, return_tensors='pt'
        )

        sample = dict(
            input_ids=token['input_ids'][0],
            attention_mask=token['attention_mask'][0]
        )
        
        label = torch.tensor(self.df.loc[idx, 'judgement'], dtype=torch.float32)
        return sample, label
        

## Model

In [10]:
class BertLstmModel(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.bert = transformers.AutoModel.from_pretrained(hps.model_name, config=bert_config)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        self.leakyrelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.regressor = nn.Linear(hidden_size*2, 1)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        self.lstm.flatten_parameters()
        out, _ = self.lstm(outputs['last_hidden_state'], None)
        out = self.leakyrelu(out)
        sequence_output = out[:, -1, :]
        output = self.dropout(sequence_output)
        logits = torch.flatten(self.regressor(output))
        return logits

In [11]:
class BertLstmExModel(nn.Module):
    def __init__(self, hidden_size, config, use_hidden_n=10):
        super().__init__()
        
        self.bert = transformers.AutoModel.from_pretrained(hps.model_name, config=bert_config)
        self.hidden_size = hidden_size
        self.use_hidden_n = use_hidden_n
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, batch_first=True, bidirectional=True)
        self.leakyrelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.conv1d = nn.Conv1d(in_channels=self.use_hidden_n, out_channels=1, kernel_size=3, padding='same')
        self.regressor = nn.Linear(self.hidden_size*2, 1)
        
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states_list = [outputs['hidden_states'][-1*i] for i in range(1, self.use_hidden_n+1)]
        self.lstm.flatten_parameters()
        out_list = [
            self.dropout(
                self.leakyrelu(
                    self.lstm(hidden_state, None)[0]
                )[:, -1, :]
            ).view(-1, 1, self.hidden_size*2)  # (batch, use_hidden_n, hidden_size*2)
        for hidden_state in hidden_states_list]

        out = torch.cat(out_list, dim=1)

        out = self.dropout(self.leakyrelu(self.conv1d(out)))
        out = out.view(out.size(0), -1)

        logits = torch.flatten(self.regressor(out))
        return logits

## Checkpoint

In [12]:
class ModelCheckpoint:
    def __init__(self, save_dir:str, save_name:str, cv_id:int):
        os.makedirs(save_dir, exist_ok=True)
        self.cv_id = cv_id
        self.save_dir = save_dir
        self.save_name = save_name
        self.best_loss = self.best_acc = self.best_fbeta_score = 0.0

    def get_checkpoint_name(self):
        checkpoint_name = f"{self.save_name.replace('/', '_')}_cv{self.cv_id}.pth"
        checkpoint_name = os.path.join(self.save_dir, checkpoint_name)
        return checkpoint_name

    def save_checkpoint(self, model):
        torch.save(model.state_dict(), self.get_checkpoint_name())

    def load_checkpoint(self, model=None, manual_name=None):
        if manual_name is None:
            checkpoint_name = self.get_checkpoint_name()
        else:
            checkpoint_name = manual_name
        print(checkpoint_name)
        model.load_state_dict(torch.load(checkpoint_name))
        return model

## Fit

In [13]:
def fit(dataloaders, model, optimizer, num_epochs, device, batch_size, lr_scheduler, cv_id):

    seed_torch(hps.random_seed)

    history = {
        'train':{'loss':[], 'acc':[], 'fbscore':[]},
        'val':{'loss':[], 'acc':[], 'fbscore':[]},
        'lr':[],
    }

    checkpoint = ModelCheckpoint(save_dir='cross_validation_weights', save_name='bert_text_classification', cv_id=cv_id)
    best_model_wts = copy.deepcopy(model.state_dict())

    print(f"Using device : {device}")
    for epoch in range(num_epochs):
        print(f"【Epoch {epoch+1: 3}/{num_epochs: 3}】   LR -> ", end='')
        for i, params in enumerate(optimizer.param_groups):
            print(f"Group{i}: {params['lr']:.4e}", end=' / ')
        print('')

        for phase in ['train', 'val']:
            running_loss = 0.0
            running_corrects = 0
            running_fbeta_score = 0.0
            if phase == 'train':
                model.train()
            else:
                model.eval()
            for i, (inputs, labels) in enumerate(tqdm(dataloaders[phase])):
                input_ids = inputs['input_ids']
                attention_mask = inputs['attention_mask']
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    logits_outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                    pos_weight = torch.tensor([hps.class_1_weight for i in range(input_ids.size(0))]).to(device)
                    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
                    loss = criterion(logits_outputs, labels)

                    outputs = torch.sigmoid(logits_outputs)
                    preds = torch.where(outputs >= 0.5, 1, 0)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        lr_scheduler.step()

                running_loss += loss.item() * input_ids.size(0)
                running_corrects += torch.sum(preds == labels)
                running_fbeta_score += fbeta_score(labels.to('cpu').detach().numpy(), preds.to('cpu').detach().numpy(), beta=7.0, zero_division=0) * input_ids.size(0)    

                if phase == 'train':
                    if i % 10 == 9:
                        total_num = float((i * batch_size) + input_ids.size(0))
                        print(f"{i+1: 4}/{len(dataloaders[phase]): 4}  <{phase}> Loss:{(running_loss/total_num):.4f}  Acc:{(running_corrects/total_num):.4f}  fbScore:{(running_fbeta_score/total_num):.4f}   LR -> ", end='')
                        for i, params in enumerate(optimizer.param_groups):
                            print(f"Group{i}: {params['lr']:.4e}", end=' / ')
                            if isinstance(optimizer.param_groups[0]['lr'], float):
                                history['lr'].append(optimizer.param_groups[0]['lr'])
                            else:
                                history['lr'].append(optimizer.param_groups[0]['lr'].item())
                        print('')

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)
            epoch_fbscore = running_fbeta_score / len(dataloaders[phase].dataset)
            
            print(f"<{phase}> Loss:{epoch_loss:.4f}  Acc:{epoch_acc:.4f}  fbScore:{epoch_fbscore:.4f}")

            history[phase]['loss'].append(epoch_loss)
            history[phase]['acc'].append(epoch_acc.item())
            history[phase]['fbscore'].append(epoch_fbscore)


            if phase == 'val' and epoch_fbscore > checkpoint.best_fbeta_score:
                print(f"Checkpoints have been updated to the epoch {epoch+1} weights.")
                checkpoint.best_loss = epoch_loss
                checkpoint.best_acc = epoch_acc
                checkpoint.best_fbeta_score = epoch_fbscore
                checkpoint.best_epoch = epoch+1
                best_model_wts = copy.deepcopy(model.state_dict())

        print('-' * 150)

    model.load_state_dict(best_model_wts)
    checkpoint.save_checkpoint(model)
    torch.cuda.empty_cache()

    return model, history

## Inference

In [14]:
def inference(model, dataloader, device, evaluate=True):
    
    running_loss = 0.0
    running_corrects = 0
    running_fbeta_score = 0.0

    preds_labels_dict = dict(preds = np.empty(0), labels = np.empty(0))

    model.eval()

    for i, (inputs, labels) in enumerate(tqdm(dataloader)):
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            logits_outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            if evaluate:
                pos_weight = torch.tensor([hps.class_1_weight for i in range(input_ids.size(0))]).to(device)
                criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
                loss = criterion(logits_outputs, labels)

            outputs = torch.sigmoid(logits_outputs)
            preds = torch.where(outputs >= 0.5, 1, 0)
            
            if evaluate:
                running_loss += loss.item() * input_ids.size(0)
                running_corrects += torch.sum(preds == labels)
                running_fbeta_score += fbeta_score(labels.to('cpu').detach().numpy(), preds.to('cpu').detach().numpy(), beta=7.0, zero_division=0) * input_ids.size(0)

            preds_labels_dict['preds']  = np.hstack([preds_labels_dict['preds'], preds.to('cpu').detach().numpy().copy()])
            preds_labels_dict['labels']  = np.hstack([preds_labels_dict['labels'], labels.to('cpu').detach().numpy().copy()])
    
    if evaluate:
        loss = running_loss / len(dataloader.dataset)
        acc = running_corrects / len(dataloader.dataset)
        fbscore = running_fbeta_score / len(dataloader.dataset)
        print(f"Loss:{loss:.4f}  Acc:{acc:.4f}  fbScore:{fbscore:.4f}")
    return preds_labels_dict

## CrossValidation Loop

In [15]:
def model_setup(model, dataloaders):

    optimizer = optim.AdamW(
        params=[
            {'params': model.bert.embeddings.parameters(), 'lr': 1e-5},
            {'params': model.bert.encoder.parameters(), 'lr': 2e-5},
            {'params': model.bert.pooler.parameters(), 'lr': 3e-5},
            {'params': model.lstm.parameters(), 'lr': 5e-4},
            {'params': model.conv1d.parameters(), 'lr': 5e-4},
            {'params': model.regressor.parameters(), 'lr': 5e-4}
        ]
    )
    num_warmup_steps = round(hps.num_epochs * len(dataloaders['train']) * 0.1)
    num_training_steps = round(hps.num_epochs * len(dataloaders['train']))
    print(f"InitLR:{hps.initial_lr} / num_warmup_steps:{num_warmup_steps} / num_training_steps:{num_training_steps}")
    lr_scheduler = transformers.get_linear_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=num_warmup_steps, 
                                                                num_training_steps=num_training_steps, last_epoch=-1)

    return (optimizer, lr_scheduler)

In [16]:
def cross_validation(cv_n, orig_df, test_df):

    logs = {
        'fit_history':[],
        'test_preds_labels':[],
        'test_fb_score':[],
    }

    for i in range(cv_n):
        print('\033[32m' + f"Cross-validation loop : {i+1}/{cv_n}" + '\033[0m')

        # DataFrame
        train_df = orig_df.loc[orig_df.cv_id != i].copy().reset_index(drop=True)
        valid_df = orig_df.loc[orig_df.cv_id == i].copy().reset_index(drop=True)
        test_df = test_df.reset_index(drop=True)
        print(f"Train  ->  label_1:{train_df.judgement.sum()} / all:{train_df.judgement.count()}   ({train_df.judgement.sum() / train_df.judgement.count() * 100:.3f}%)")
        print(f"Valid  ->  label_1:{valid_df.judgement.sum()} / all:{valid_df.judgement.count()}   ({valid_df.judgement.sum() / valid_df.judgement.count() * 100:.3f}%)")

        # Dataset / Dataloader
        phase_param = {
            "df":{'train': train_df, 'val': valid_df, 'test': test_df, 'submit': submit_df},
            "argument":{'train': hps.train_argument, 'val': False, 'test': False, 'submit': False},
            "batch_size":{'train':hps.batch_size, 'val':hps.batch_size*4, 'test':hps.batch_size*4, 'submit': hps.batch_size*4},
            "shuffle":{'train': True, 'val': False, 'test': False, 'submit': False},
            "upsample_pos_n":{'train': hps.upsample_pos_n, 'val': 1, 'test': 1, 'submit': 1},
        }
        datasets = {phase:TextClassificationDataset(df=phase_param['df'][phase], tokenizer=base_tokenizer, use_col=hps.use_col,\
                                                    token_max_length=hps.token_max_length, argument=phase_param['argument'][phase],\
                                                    upsample_pos_n=phase_param['upsample_pos_n'][phase]) for phase in ['train', 'val', 'test', 'submit']}
        dataloaders = {phase: DataLoader(datasets[phase], batch_size=phase_param['batch_size'][phase], \
                                        shuffle=phase_param['shuffle'][phase]) for phase in ['train', 'val', 'test', 'submit']}
        
        # Model / Optimizer
        if hps.model_type == 'lstm':
            print(f"Choosed BertLstmModel")
            model = BertLstmModel(hidden_size=bert_config.hidden_size)
        elif hps.model_type == 'lstm_ex':
            print(f"Choosed BertLstmExModel")
            model = BertLstmExModel(hidden_size=bert_config.hidden_size, config=bert_config, use_hidden_n=4)

        optimizer, lr_scheduler = model_setup(model, dataloaders)
        model = model.to(device)
        device_num = torch.cuda.device_count()
        if device_num > 1:
            print(f"Use {device_num} GPUs")
            model = nn.DataParallel(model)

        # Training / Validation
        model, fit_history = fit(dataloaders=dataloaders, model=model, optimizer=optimizer, num_epochs=hps.num_epochs, 
                             device=device, batch_size=hps.batch_size, lr_scheduler=lr_scheduler, cv_id=i)

        # Evaluate
        print(f"Evaluate Test Dataset")
        test_preds_labels_dict = inference(model, dataloader=dataloaders['test'], device=device)
        test_fb_score = fbeta_score(y_true=test_preds_labels_dict['labels'], y_pred=test_preds_labels_dict['preds'], beta=7.0)
        print(f"fb_score : {test_fb_score}")   

        logs['fit_history'].append(fit_history)
        logs['test_preds_labels'].append(test_preds_labels_dict)
        logs['test_fb_score'].append(test_fb_score)

        del model, datasets, dataloaders
        torch.cuda.empty_cache()
        print()

    return logs


In [ ]:
logs = cross_validation(cv_n=hps.cv_n, orig_df=train_df, test_df=test_df)

In [17]:
logs.keys()

dict_keys(['fit_history', 'test_preds_labels', 'test_fb_score'])

In [18]:
logs['test_fb_score'][1]

0.9067357512953368

In [19]:
for i in range(hps.cv_n):
    fb_score = logs['test_fb_score'][i]
    print(fb_score)

0.9028831562974203
0.9067357512953368
0.884621200665356
0.892047172664046
0.8957654723127035


In [20]:
test_pred_df = pd.DataFrame(columns=[f"cv{i}" for i in range(hps.cv_n)] + ['cv_ensemble', 'label'])
test_pred_df['label'] = logs['test_preds_labels'][0]['labels']
for i in range(hps.cv_n):
    test_pred_df[f"cv{i}"] = logs['test_preds_labels'][i]['preds']




test_pred_df['cv_ensemble'] = test_pred_df.loc[:, 'cv0':'cv4'].mean(axis=1).map(lambda x: 1 if x >= 0.5 else 0)


display(test_pred_df)
display(test_pred_df.describe())

,cv0,cv1,cv2,cv3,cv4,cv_ensemble,label
0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,0.0,0.0,0.0,0.0,0.0,0,0.0
2,0.0,0.0,0.0,0.0,0.0,0,0.0
3,0.0,0.0,0.0,0.0,0.0,0,0.0
4,0.0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...
5424,0.0,0.0,0.0,0.0,0.0,0,0.0
5425,0.0,0.0,0.0,0.0,1.0,0,0.0
5426,0.0,0.0,0.0,0.0,0.0,0,0.0
5427,0.0,0.0,0.0,0.0,0.0,0,0.0


,cv0,cv1,cv2,cv3,cv4,cv_ensemble,label
count,5429.000000,5429.000000,5429.000000,5429.000000,5429.000000,5429.000000,5429.000000
mean,0.076626,0.071468,0.080862,0.081046,0.106834,0.076441,0.023209
std,0.266021,0.257629,0.272648,0.272931,0.308930,0.265727,0.150580
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
cv_ensemble_fb_score = fbeta_score(y_true=test_pred_df['label'], y_pred=test_pred_df['cv_ensemble'], beta=7.0)
print(f"CV_Ensemble_Fb_score : {cv_ensemble_fb_score}")

# Submit

In [18]:
class Predicting:
    def __init__(self, weights_dir:str):
        self.weights_dir = weights_dir
        self.logs = {'test_preds_labels':[]}
        self.pred_df = pd.DataFrame(columns=[f"cv{i}" for i in range(hps.cv_n)] + ['cv_ensemble', 'label'])

    def predict(self, df, pred_phase='submit'):

        df.reset_index(drop=False, inplace=True)

        weights_path_list = os.listdir(self.weights_dir)
        for weights_path in weights_path_list:
            print('\033[32m' + weights_path + '\033[0m')

            # load model
            if hps.model_type == 'lstm':
                print(f"Choosed BertLstmModel")
                model = BertLstmModel(hidden_size=bert_config.hidden_size)
            elif hps.model_type == 'lstm_ex':
                print(f"Choosed BertLstmExModel")
                model = BertLstmExModel(hidden_size=bert_config.hidden_size, config=bert_config, use_hidden_n=4)
            
            # To GPU
            print(f"Model to GPU ... ", end='')
            model = model.to(device)
            device_num = torch.cuda.device_count()
            if device_num > 1:
                print(f"Use {device_num} GPUs ", end='')
                model = nn.DataParallel(model)
            print(f"Done!")

            # load
            print(f"Loading model ... ", end='')
            model.load_state_dict(torch.load(os.path.join(self.weights_dir, weights_path)))
            print(f"Done!")

            # Datasets / Dataloaders

            phase_param = {
                "argument":{'test': False, 'submit': False},
                "batch_size":{'test':hps.batch_size*4, 'submit': hps.batch_size*4},
                "shuffle":{'test': False, 'submit': False},
                "upsample_pos_n":{'val': 1, 'test': 1, 'submit': 1},
            }
            datasets = {phase:TextClassificationDataset(df=df, tokenizer=base_tokenizer, use_col=hps.use_col,\
                                                        token_max_length=hps.token_max_length, argument=phase_param['argument'][phase],\
                                                        upsample_pos_n=phase_param['upsample_pos_n'][phase]) for phase in ['test', 'submit']}
            dataloaders = {phase: DataLoader(datasets[phase], batch_size=phase_param['batch_size'][phase], \
                                            shuffle=phase_param['shuffle'][phase]) for phase in ['test', 'submit']}


            # inference
            preds_labels_dict = inference(model, dataloader=dataloaders[pred_phase], device=device, evaluate=False)
            self.logs['test_preds_labels'].append(preds_labels_dict)

            del model, datasets, dataloaders
            torch.cuda.empty_cache()
            print()

    def get_logs(self):
        return self.logs

    def ensemble(self):
        
        self.pred_df['label'] = self.logs['test_preds_labels'][0]['labels']
        for i in range(hps.cv_n):
            self.pred_df[f"cv{i}"] = self.logs['test_preds_labels'][i]['preds']
        self.pred_df['cv_ensemble'] = self.pred_df.loc[:, 'cv0':'cv4'].mean(axis=1).map(lambda x: 1 if x >= 0.5 else 0)
        return self.pred_df

    def get_fb_score(self):
        cv_ensemble_fb_score = fbeta_score(y_true=self.pred_df['label'], y_pred=self.pred_df['cv_ensemble'], beta=7.0)
        return cv_ensemble_fb_score


In [19]:
predict = Predicting(weights_dir='cross_validation_weights')

In [ ]:
predict.predict(df=submit_df.copy(), pred_phase='submit')

In [44]:
pred_df = predict.ensemble()

In [49]:
pred_df

,cv0,cv1,cv2,cv3,cv4,cv_ensemble,label
27145,0.0,0.0,0.0,0.0,0.0,0,-1.0
27146,0.0,0.0,0.0,0.0,0.0,0,-1.0
27147,0.0,0.0,0.0,0.0,0.0,0,-1.0
27148,0.0,0.0,0.0,0.0,0.0,0,-1.0
27149,0.0,0.0,0.0,0.0,0.0,0,-1.0
...,...,...,...,...,...,...,...
67974,0.0,0.0,0.0,0.0,0.0,0,-1.0
67975,0.0,0.0,0.0,0.0,0.0,0,-1.0
67976,0.0,0.0,0.0,0.0,0.0,0,-1.0
67977,0.0,0.0,0.0,0.0,0.0,0,-1.0


In [54]:
pred_df.describe()

,cv0,cv1,cv2,cv3,cv4,cv_ensemble,label
count,40834.000000,40834.000000,40834.000000,40834.000000,40834.000000,40834.000000,40834.0
mean,0.082750,0.063379,0.074766,0.078415,0.080595,0.072978,-1.0
std,0.275507,0.243646,0.263017,0.268827,0.272215,0.260104,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.0


In [48]:
pred_df.index += len(orig_df)

In [50]:
sample_submit_df['judgement'] = pred_df['cv_ensemble']

In [51]:
sample_submit_df

,judgement
27145,0
27146,0
27147,0
27148,0
27149,0
...,...
67974,0
67975,0
67976,0
67977,0


In [53]:
jst = dt.timezone(dt.timedelta(hours=+9), 'JST')
dt_now = dt.datetime.now(jst)
dt_now_str = dt_now.strftime('%Y%m%d_%H%M')
submit_str = f"{dt_now_str}_{cv_ensemble_fb_score:.4f}".replace('.', '-')
submit_str = f"{submit_str}.csv"
print(submit_str)

os.makedirs('./submit', exist_ok=True)
sample_submit_df.to_csv(os.path.join('submit', submit_str), header=False)

20211002_0045_0-0000.csv
